<div style="background-color:#e9eeef; border-right: 10px solid #36b38d ;" height=300 width=100%>
    <font color=#2b3444 size=8 align=left ><strong> Predict Lead Conversion</strong></font><br>
    <font color=#36b38d size=6 align=left ><strong> Model Training Interface</strong></font>
</div>
 



In [49]:
import ipywidgets as widgets
import pandas as pd
from ipywidgets import FileUpload
import base64
from IPython.display import HTML
import time
from ipywidgets import IntProgress
from IPython.display import display
import sklearn
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import xgboost as xgb
from xgboost import XGBRegressor
# import pandas as pd
import numpy as np
import datetime
import re
# import time
import pickle
# import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 10)
import voila
import io
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.model_selection import train_test_split
# import codecs

# XGBoost Model 

_______________________________________________________________________________________________________________________________
## Steps to Generate Predictions
### - Select Pre existing or re-train a new model
**You can choose to generate predictions with the existing model. Or you can choose to re-train the model with a new dataset.**

**For Existing Model: Each prediction cycle takes a single .csv or .xlsx test file and latest version of PD Sync.The test file MUST contain the month and year columns. If you would like to predict for multiple months, please combine all the months into a single file or predict for one month at a time.**

**For example if you would like to generate predictions for 2 months - Nov and Dec. You can either combine the data for both the months in a single file or upload two separate files containing data for Nov and Dec respectively.**

**For Re-training: Each prediction cycle takes two files, pd sync .xlsx/.csv for training, and a test  .xlsx or .csv file.**


### - Select level of training / testing
**What are Levels?**

**There are 2 levels within which the model can generate predictions. These are - L0, an L1 GCP**


### - Format and Predict
**This stage is where all the uploaded files are formatted according to the level selected and then fed into the model.**

**The model along with the rest of the code runs in the backend and generates predictions.**

**Expected result-**

**Predicted number of closed deals, predicted number of won deals, predicted deal duration (avg) and predicted OBV value for won deals**

**Predicted data on deal level also available for download.**
_______________________________________________________________________________________________________________________________ 

In [61]:
#-----------------------------# pd sync file upload button #-----------------------------#
pdsync_upload_button = widgets.Button(description='Next',button_style='info',tooltip='Run')

label_pdsync_test=widgets.Label('Please upload PD Sync file in .csv or .xlsx format. Please click the Next button after Upload changes from 0 to 1. **This file is not used for training')
uploader_pdsync_file = widgets.FileUpload(multiple=False)

def get_pdsync_file(b):
    input_file = list(uploader_pdsync_file.value.values())[0]
    content = input_file['content']
    #content = io.StringIO(content.decode('utf-8'))
    
    input_file_Name=input_file['metadata']['name']
    File_name_temp1=input_file_Name.split('.')
    File_type=File_name_temp1[1]
    if File_type=='csv':
        df_pdsync = pd.read_csv(input_file_Name)
#         display(df_pdsync)
    else:
        df_pdsync = pd.read_excel(content) 
#         display(df_pdsync)

#     display(df_pdsync_test)
    
    
    get_pdsync_file.data = df_pdsync
    
#----------------------------------# Test file Upload #----------------------------#
testfile_upload_button = widgets.Button(description='Next',button_style='info',tooltip='Run')

label_test=widgets.Label('Please upload test file in .csv or .xlsx format. Please click the Next button after Upload changes from 0 to 1.')
uploader_test_file = widgets.FileUpload(accept='*.csv',multiple=False)

def get_test_file(b):
    input_file = list(uploader_test_file.value.values())[0]
    content = input_file['content']
#     content = io.StringIO(content.decode('utf-8'))
    input_file_Name=input_file['metadata']['name']
    File_name_temp1=input_file_Name.split('.')
    File_type=File_name_temp1[1]
    if File_type=='csv':
        df_test = pd.read_csv(input_file_Name)
#         display(df_test)
    else:
        df_test_file = pd.read_excel(content) 
#         display(df_test)

# #     display(df_pdsync_test)

    get_test_file.data = df_test
    
#--------------------------------# Create downlod links L0 pre-existing #-------------------------#
def create_download_link_l0_formatted( df, title = "Download formatted test CSV file for L0", filename = "formatted-data-L0.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'     
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

def create_download_link_l0_pred( df, title = "Download L0 prediction CSV file", filename = "prediction-L0-pretrained.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'     
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)
#--------------------------------#L0 Format for pre-existing model#-------------------------------#

def new_date(row):
    return pd.to_datetime(row['deal_created'])+pd.DateOffset(days=int(row['predicted_days']))

def l0_format_pretrain():
    
    df_pdsync = get_pdsync_file.data
    df_test = get_test_file.data
    df_test = df_test.loc[(df_test['pipeline'] == 'Opportunities')]
    df_test=df_test.loc[(df_test['channel_influence'] != 'HCLS-Alpha')]
    df_test=df_test.loc[(df_test['stage_id'].isna()==False)]
    df_test=df_test.loc[(df_test['stage_id']!='deleted')]
    df_test=df_test.loc[(df_test['stage_id']!='DA')]
    index_stage = df_test['stage_id'].isin(['On Hold'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['stage_id']] = 'onhold'
    index_stage = df_test['last_stage_id'].isin(['On Hold'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'onhold'

    index_stage = df_test['last_stage_id'].isin(['On HOLD'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'onhold'
    index_stage = df_test['last_stage_id'].isin(['Discovery Meeting'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'discovery_meeting'

    index_stage = df_test['last_stage_id'].isin(['Problem Definition Finalized'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'problem_definition_finalized'

    index_stage = df_test['last_stage_id'].isin(['Proposal Shared'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'proposal_shared'

    index_stage = df_test['last_stage_id'].isin(['Pre-Commit'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'pre_Commit'

    index_stage = df_test['last_stage_id'].isin(['Committed'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'committed'
    
    
    df_test['close_time'] = df_test['close_time'].astype('datetime64[ns]')
    df_test['deal_created'] = df_test['deal_created'].astype('datetime64[ns]')
    df_test['update_time'] = df_test['update_time'].astype('datetime64[ns]')
    df_test['last_stage_change_time'] = df_test['last_stage_change_time'].astype('datetime64[ns]')
    df_test['deal_duration'] = df_test['close_time'] -  df_test['deal_created']
    df_test['deal_duration_days'] = (df_test['deal_duration']).dt.days
    df_test['deal_value'] = pd.to_numeric(df_test['deal_value'],errors = 'coerce')
    df_test['done_activities_count'] = pd.to_numeric(df_test['done_activities_count'],errors = 'coerce')
    df_test['onhold'] = pd.to_numeric(df_test['onhold'],errors = 'coerce')
    df_test['discovery_meeting'] = pd.to_numeric(df_test['discovery_meeting'],errors = 'coerce')
    df_test['problem_definition_finalized'] = pd.to_numeric(df_test['problem_definition_finalized'],errors = 'coerce')
    df_test['proposal_shared'] = pd.to_numeric(df_test['proposal_shared'],errors = 'coerce')
    df_test['contracting'] = pd.to_numeric(df_test['contracting'],errors = 'coerce')
    df_test['committed'] = pd.to_numeric(df_test['committed'],errors = 'coerce')
    df_test['last_updated_by'] = pd.to_numeric(df_test['last_updated_by'],errors = 'coerce')
    df_test['weights'] = pd.to_numeric(df_test['weights'],errors = 'coerce')
    df_test['deal_duration_days'] = pd.to_numeric(df_test['deal_duration_days'],errors = 'coerce')
    
    
    df_test['last_stage_change_time_year'] = df_test['last_stage_change_time'].dt.year
    df_test['last_stage_change_time_month'] = df_test['last_stage_change_time'].dt.month
    df_test['last_stage_change_time_week'] = df_test['last_stage_change_time'].dt.week
    df_test['last_stage_change_time_day'] = df_test['last_stage_change_time'].dt.day
    df_test['update_time_year'] = df_test['update_time'].dt.year
    df_test['update_time_month'] = df_test['update_time'].dt.month
    df_test['update_time_week'] = df_test['update_time'].dt.week
    df_test['update_time_day'] = df_test['update_time'].dt.day
    df_test['deal_created_year'] = df_test['deal_created'].dt.year
    df_test['deal_created_month'] = df_test['deal_created'].dt.month
    df_test['deal_created_week'] = df_test['deal_created'].dt.week
    df_test['deal_created_day'] = df_test['deal_created'].dt.day
    
    conditions = [
    (df_test['stage_id'] == 'Discovery Meeting') & (df_test['last_stage_id'].isin(['discovery_meeting', 'proposal_shared','Contracting', 'problem_definition_finalized', 'pre_Commit'])),
    (df_test['stage_id'] == 'Discovery Meeting') & (~df_test['last_stage_id'].isin(['discovery_meeting', 'proposal_shared','Contracting', 'problem_definition_finalized', 'pre_Commit'])),
    (df_test['stage_id'] == 'Problem Definition Finalized') & (df_test['last_stage_id'].isin(['proposal_shared','Contracting', 'problem_definition_finalized', 'pre_Commit'])),
    (df_test['stage_id'] == 'Problem Definition Finalized') & (~df_test['last_stage_id'].isin(['proposal_shared','Contracting', 'problem_definition_finalized', 'pre_Commit'])),
    (df_test['stage_id'] == 'Proposal Shared') & (df_test['last_stage_id'].isin(['proposal_shared','Contracting', 'pre_Commit'])),
    (df_test['stage_id'] == 'Proposal Shared') & (~df_test['last_stage_id'].isin(['proposal_shared','Contracting', 'pre_Commit'])),
    (df_test['stage_id'] == 'Contracting') & (df_test['last_stage_id'].isin(['Contracting', 'pre_Commit'])),
    (df_test['stage_id'] == 'Contracting') & (~df_test['last_stage_id'].isin(['Contracting', 'pre_Commit'])),
    (df_test['stage_id'] == 'Pre-Commit') & (df_test['last_stage_id'].isin(['pre_Commit'])),
    (df_test['stage_id'] == 'Pre-Commit') & (~df_test['last_stage_id'].isin(['pre_Commit'])),
     ]

    st_values = [1,0,1,0,1,0,1,0,1,0]
    
    
    df_test['stage_id_mismatch'] = np.select(conditions, st_values)
    
    df_final = df_test[['service_line', 'platform', 'source_of_lead', 'mega_deal',
       'update_time_week', 'last_stage_change_time_week', 'update_time_day',
       'industry', 'onhold', 'BU', 'update_time_year', 'region',
       'deal_created_year', 'stage_id_mismatch', 'channel_influence',
       'committed', 'business_category', 'last_stage_change_time_month',
       'last_stage_change_time_year', 'contracting', 'stage_id',
       'update_time_month', 'last_stage_change_time_day',
       'done_activities_count', 'deal_created_month', 'deal_value',
       'deal_duration_days','deal_id','status']]
    
    df_final = df_final[df_final['status'] == 'open']
    
    cat_col = list(df_final.select_dtypes(include='O').columns)
    
    df_final[cat_col]=df_final[cat_col].fillna(df_final.mode().iloc[0])
    
    labelencoder = LabelEncoder()


    df_final[cat_col] = df_final[cat_col].apply(LabelEncoder().fit_transform)

    df_final = df_final.astype('float64')
    
    
    deal_id = df_final['deal_id']
    status = df_final['status']
    
    display(create_download_link_l0_formatted(df_final))
    
    df_final = df_final.drop(columns=['deal_id', 'status'])
    
    pred_classification = l0_predict(df_final)
    
    df_final['class_pred'] = pred_classification
    
    deal_duration_days = df_final['deal_duration_days']
    
    df_final = df_final.drop(columns=['deal_duration_days'])
    
    pred_regression = l0_predict_reg(df_final)
    
    df_final['predicted_days'] = pred_regression
    
    display(df_final)
    
    df_final['deal_id'] = deal_id
    
    df = pd.merge(df_final, df_test[["deal_id","deal_created", "sow_start_date", "sow_end_date"]], on="deal_id", how="right")
    
    df = df.dropna() 

    df['predicted_close_date']= df.apply(new_date, axis=1)
    
    df['predicted_close_monthyear'] = df.predicted_close_date.dt.to_period('M')
    
    df_csv = df[['deal_id', 'deal_value', 'deal_created', 'sow_start_date', 'sow_end_date','channel_influence', 'class_pred', 'predicted_close_date']]
    
    display(create_download_link_l0_pred(df_csv))
    
    df_date = df[(df['predicted_close_monthyear'] > "2022-09")]
    
    deal_value_monthly =  df_date.groupby(['predicted_close_monthyear'], as_index=False)['deal_value'].sum()
    print('Predicted OBV for all deals month-wise:\n')
    time.sleep(2)
    display(deal_value_monthly)
    df_date_won = df_date[df_date['class_pred']==1.0]
    deal_value_won =  df_date_won.groupby(['predicted_close_monthyear','class_pred'], as_index=False)['deal_value'].sum()
    print('Predicted OBV for predicted won deals month-wise:\n')
    time.sleep(2)
    display(deal_value_won)
    won_deals = df_date_won.groupby('predicted_close_monthyear', as_index=False)['class_pred'].count()
    print('Predicted no. of won deals month-wise:\n')
    time.sleep(2)
    display(won_deals)
    deal_duration = df_date.groupby('predicted_close_monthyear', as_index=False)['predicted_days'].mean()
    print('Predicted avg. deal duration of all deals month-wise:\n')
    time.sleep(2)
    display(deal_duration)
    deal_duration_won = df_date_won.groupby('predicted_close_monthyear', as_index=False)['predicted_days'].mean()
    print('Predicted avg. deal duration of won deals month-wise:\n')
    time.sleep(2)
    display(deal_duration_won)
    
    

    

#-------------------------# Load pre-trained model L0 #---------------------------------------#
def l0_model():
    with open('./xgb_classification.pkl', 'rb') as f:
        l0_model = pickle.load(f)
    return l0_model


def l0_predict(df):
         # Feature arragend as per the model
    model_l0=l0_model()
    predict_l0 = model_l0.predict(df)               # Prediction using the model
#     prediction = np.sum(predict_l0, dtype=np.int32)      # Predicted result is made into readable format
    return predict_l0
 
def l0_model_reg():
    with open('./xgb_regression.pkl', 'rb') as f:
        l0_model_reg = pickle.load(f)
    return l0_model_reg


def l0_predict_reg(df):
         # Feature arragend as per the model
    model_l0_reg=l0_model_reg()
    predict_l0_reg = model_l0_reg.predict(df)               # Prediction using the model
#     prediction = np.sum(predict_l0, dtype=np.int32)      # Predicted result is made into readable format
    return predict_l0_reg    
    
#--------------------------------# Create downlod links L0 retraining #-------------------------#
def create_download_link_l0_retrain( df, title = "Download formatted test CSV file for L0", filename = "formatted-data-L0.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'     
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

def create_download_link_l0_pred_rt( df, title = "Download L0 prediction CSV file", filename = "prediction-L0-pretrained.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'     
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)
#--------------------------------#L0 Format for re-training model#-------------------------------#

def new_date(row):
    return pd.to_datetime(row['deal_created'])+pd.DateOffset(days=int(row['predicted_days']))

def l0_format_retrain():
    
    df_pdsync = get_pdsync_file.data
    df_test = get_test_file.data
    df_test = df_test.loc[(df_test['pipeline'] == 'Opportunities')]
    df_test=df_test.loc[(df_test['channel_influence'] != 'HCLS-Alpha')]
    df_test=df_test.loc[(df_test['stage_id'].isna()==False)]
    df_test=df_test.loc[(df_test['stage_id']!='deleted')]
    df_test=df_test.loc[(df_test['stage_id']!='DA')]
    index_stage = df_test['stage_id'].isin(['On Hold'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['stage_id']] = 'onhold'
    index_stage = df_test['last_stage_id'].isin(['On Hold'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'onhold'

    index_stage = df_test['last_stage_id'].isin(['On HOLD'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'onhold'
    index_stage = df_test['last_stage_id'].isin(['Discovery Meeting'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'discovery_meeting'

    index_stage = df_test['last_stage_id'].isin(['Problem Definition Finalized'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'problem_definition_finalized'

    index_stage = df_test['last_stage_id'].isin(['Proposal Shared'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'proposal_shared'

    index_stage = df_test['last_stage_id'].isin(['Pre-Commit'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'pre_Commit'

    index_stage = df_test['last_stage_id'].isin(['Committed'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'committed'
    
    
    df_test['close_time'] = df_test['close_time'].astype('datetime64[ns]')
    df_test['deal_created'] = df_test['deal_created'].astype('datetime64[ns]')
    df_test['update_time'] = df_test['update_time'].astype('datetime64[ns]')
    df_test['last_stage_change_time'] = df_test['last_stage_change_time'].astype('datetime64[ns]')
    df_test['deal_duration'] = df_test['close_time'] -  df_test['deal_created']
    df_test['deal_duration_days'] = (df_test['deal_duration']).dt.days
    df_test['deal_value'] = pd.to_numeric(df_test['deal_value'],errors = 'coerce')
    df_test['done_activities_count'] = pd.to_numeric(df_test['done_activities_count'],errors = 'coerce')
    df_test['onhold'] = pd.to_numeric(df_test['onhold'],errors = 'coerce')
    df_test['discovery_meeting'] = pd.to_numeric(df_test['discovery_meeting'],errors = 'coerce')
    df_test['problem_definition_finalized'] = pd.to_numeric(df_test['problem_definition_finalized'],errors = 'coerce')
    df_test['proposal_shared'] = pd.to_numeric(df_test['proposal_shared'],errors = 'coerce')
    df_test['contracting'] = pd.to_numeric(df_test['contracting'],errors = 'coerce')
    df_test['committed'] = pd.to_numeric(df_test['committed'],errors = 'coerce')
    df_test['last_updated_by'] = pd.to_numeric(df_test['last_updated_by'],errors = 'coerce')
    df_test['weights'] = pd.to_numeric(df_test['weights'],errors = 'coerce')
    df_test['deal_duration_days'] = pd.to_numeric(df_test['deal_duration_days'],errors = 'coerce')
    
    
    df_test['last_stage_change_time_year'] = df_test['last_stage_change_time'].dt.year
    df_test['last_stage_change_time_month'] = df_test['last_stage_change_time'].dt.month
    df_test['last_stage_change_time_week'] = df_test['last_stage_change_time'].dt.week
    df_test['last_stage_change_time_day'] = df_test['last_stage_change_time'].dt.day
    df_test['update_time_year'] = df_test['update_time'].dt.year
    df_test['update_time_month'] = df_test['update_time'].dt.month
    df_test['update_time_week'] = df_test['update_time'].dt.week
    df_test['update_time_day'] = df_test['update_time'].dt.day
    df_test['deal_created_year'] = df_test['deal_created'].dt.year
    df_test['deal_created_month'] = df_test['deal_created'].dt.month
    df_test['deal_created_week'] = df_test['deal_created'].dt.week
    df_test['deal_created_day'] = df_test['deal_created'].dt.day
    
    conditions = [
    (df_test['stage_id'] == 'Discovery Meeting') & (df_test['last_stage_id'].isin(['discovery_meeting', 'proposal_shared','Contracting', 'problem_definition_finalized', 'pre_Commit'])),
    (df_test['stage_id'] == 'Discovery Meeting') & (~df_test['last_stage_id'].isin(['discovery_meeting', 'proposal_shared','Contracting', 'problem_definition_finalized', 'pre_Commit'])),
    (df_test['stage_id'] == 'Problem Definition Finalized') & (df_test['last_stage_id'].isin(['proposal_shared','Contracting', 'problem_definition_finalized', 'pre_Commit'])),
    (df_test['stage_id'] == 'Problem Definition Finalized') & (~df_test['last_stage_id'].isin(['proposal_shared','Contracting', 'problem_definition_finalized', 'pre_Commit'])),
    (df_test['stage_id'] == 'Proposal Shared') & (df_test['last_stage_id'].isin(['proposal_shared','Contracting', 'pre_Commit'])),
    (df_test['stage_id'] == 'Proposal Shared') & (~df_test['last_stage_id'].isin(['proposal_shared','Contracting', 'pre_Commit'])),
    (df_test['stage_id'] == 'Contracting') & (df_test['last_stage_id'].isin(['Contracting', 'pre_Commit'])),
    (df_test['stage_id'] == 'Contracting') & (~df_test['last_stage_id'].isin(['Contracting', 'pre_Commit'])),
    (df_test['stage_id'] == 'Pre-Commit') & (df_test['last_stage_id'].isin(['pre_Commit'])),
    (df_test['stage_id'] == 'Pre-Commit') & (~df_test['last_stage_id'].isin(['pre_Commit'])),
     ]

    st_values = [1,0,1,0,1,0,1,0,1,0]
    
    
    df_test['stage_id_mismatch'] = np.select(conditions, st_values)
    
    df_final = df_test[['service_line', 'platform', 'source_of_lead', 'mega_deal',
       'update_time_week', 'last_stage_change_time_week', 'update_time_day',
       'industry', 'onhold', 'BU', 'update_time_year', 'region',
       'deal_created_year', 'stage_id_mismatch', 'channel_influence',
       'committed', 'business_category', 'last_stage_change_time_month',
       'last_stage_change_time_year', 'contracting', 'stage_id',
       'update_time_month', 'last_stage_change_time_day',
       'done_activities_count', 'deal_created_month', 'deal_value',
       'deal_duration_days','deal_id','status']]
    
    df_open = df_test[['service_line', 'platform', 'source_of_lead', 'mega_deal',
       'update_time_week', 'last_stage_change_time_week', 'update_time_day',
       'industry', 'onhold', 'BU', 'update_time_year', 'region',
       'deal_created_year', 'stage_id_mismatch', 'channel_influence',
       'committed', 'business_category', 'last_stage_change_time_month',
       'last_stage_change_time_year', 'contracting', 'stage_id',
       'update_time_month', 'last_stage_change_time_day',
       'done_activities_count', 'deal_created_month', 'deal_value',
       'deal_duration_days','deal_id','status']]

    
    df_final = df_final[~(df_final['status'] == 'open')]
    
    df_open = df_open[df_open['status'] == 'open']
    
    cat_col = list(df_final.select_dtypes(include='O').columns)
    
    df_final[cat_col]=df_final[cat_col].fillna(df_final.mode().iloc[0])
    
    df_open[cat_col]=df_open[cat_col].fillna(df_open.mode().iloc[0])
    
    labelencoder = LabelEncoder()


    df_final[cat_col] = df_final[cat_col].apply(LabelEncoder().fit_transform)
    
    df_open[cat_col] = df_open[cat_col].apply(LabelEncoder().fit_transform)

    df_final = df_final.astype('float64')
    
    df_open = df_open.astype('float64')
    
    deal_id = df_final['deal_id']
    
    deal_id_open = df_open['deal_id']
    
    status = df_final['status']
    
    
    display(create_download_link_l0_retrain(df_final))
    
    df_final = df_final.drop(columns=['deal_id'])
    
    df_open = df_open.drop(columns=['deal_id','status'])
    
    X = df_final.drop(columns=['status'])
    
    y = df_final['status']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

#     estimator = XGBClassifier(
#     objective= 'binary:logistic',
#     seed=42
#     )
    
#     params = {
#         'min_child_weight': [1,3, 5, 10],
#         'gamma': [0.5, 1, 1.5, 2, 5],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 1.0],
#         'max_depth': [2,3, 4, 5],
#         'learning_rate':[0.1, 0.01, 0.05, 0.001]
#         'n_estimators':[100,200,300,500,700,1000]
#         }
    
#     grid_search = GridSearchCV(
#     estimator=estimator,
#     param_grid=parameters,
#     scoring = 'roc_auc',
#     verbose=True
#     )
    
#     gcv_class = grid_search.fit(X, Y)
#     xgb_class = gcv.best_estimator_
    
    xgbmodel = XGBClassifier(learning_rate = 0.01, n_estimators=300, max_depth=5)
    xgbmodel.fit(X_train, y_train)
    
    pred_open = xgbmodel.predict(df_open)
    
    
    df_open['class_pred'] = pred_open
    
    deal_duration_days = df_final['deal_duration_days']
    
    X_reg = df_final.drop(columns=['deal_duration_days'])
    
    Y_reg = df_final['deal_duration_days']
    
    
    X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, Y_reg, test_size=0.10)
    
    reg_model = xgb.XGBRegressor(colsample_bytree = 0.7, learning_rate=0.05, max_depth=3, n_estimators= 1000)
    
    reg_model.fit(X_train_reg, y_train_reg)
    
    df_open = df_open.drop(columns=['deal_duration_days'])
    
    df_open= df_open.rename(columns={'class_pred': 'status'})
    
    df_open = df_open[['service_line', 'platform', 'source_of_lead', 'mega_deal',
       'update_time_week', 'last_stage_change_time_week', 'update_time_day',
       'industry', 'onhold', 'BU', 'update_time_year', 'region',
       'deal_created_year', 'stage_id_mismatch', 'channel_influence',
       'committed', 'business_category', 'last_stage_change_time_month',
       'last_stage_change_time_year', 'contracting', 'stage_id',
       'update_time_month', 'last_stage_change_time_day',
       'done_activities_count', 'deal_created_month', 'deal_value',
       'status']]
    
    pred_open_reg = reg_model.predict(df_open)
    
    df_open['predicted_days'] = pred_open_reg
    
    display(df_open)
    
    
    
    df_open['deal_id'] = deal_id_open
    
    df = pd.merge(df_open, df_test[["deal_id","deal_created", "sow_start_date", "sow_end_date"]], on="deal_id", how="right")
    
    df = df.dropna() 

    df['predicted_close_date']= df.apply(new_date, axis=1)
    
    df['predicted_close_monthyear'] = df.predicted_close_date.dt.to_period('M')
    
    df= df.rename(columns={'status': 'class_pred'})
    
    df_csv = df[['deal_id', 'deal_value', 'deal_created', 'sow_start_date', 'sow_end_date','channel_influence', 'class_pred', 'predicted_close_date']]
    
    display(create_download_link_l0_pred_rt(df_csv))
    
    df_date = df[(df['predicted_close_monthyear'] > "2022-09")]
    
    deal_value_monthly =  df_date.groupby(['predicted_close_monthyear'], as_index=False)['deal_value'].sum()
    print('Predicted OBV for all deals month-wise:\n')
    time.sleep(2)
    display(deal_value_monthly)
    df_date_won = df_date[df_date['class_pred']==1.0]
    deal_value_won =  df_date_won.groupby(['predicted_close_monthyear','class_pred'], as_index=False)['deal_value'].sum()
    print('Predicted OBV for predicted won deals month-wise:\n')
    time.sleep(2)
    display(deal_value_won)
    won_deals = df_date_won.groupby('predicted_close_monthyear', as_index=False)['class_pred'].count()
    print('Predicted no. of won deals month-wise:\n')
    time.sleep(2)
    display(won_deals)
    deal_duration = df_date.groupby('predicted_close_monthyear', as_index=False)['predicted_days'].mean()
    print('Predicted avg. deal duration of all deals month-wise:\n')
    time.sleep(2)
    display(deal_duration)
    deal_duration_won = df_date_won.groupby('predicted_close_monthyear', as_index=False)['predicted_days'].mean()
    print('Predicted avg. deal duration of won deals month-wise:\n')
    time.sleep(2)
    display(deal_duration_won)
     
#--------------------------------# Create downlod links L1-GCP pre-existing #-------------------------#
def create_download_link_l1gcp_formatted( df, title = "Download formatted test CSV file for L1-GCP", filename = "formatted-data-L1-GCP.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'     
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

def create_download_link_l1gcp_pred( df, title = "Download L1-GCP prediction CSV file", filename = "prediction-L1GCP-pretrained.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'     
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

#--------------------------------#L1-GCP Format for pre-existing model#-------------------------------#

def new_date(row):
    return pd.to_datetime(row['deal_created'])+pd.DateOffset(days=int(row['predicted_days']))

def l1gcp_format_pretrain():
    
    df_pdsync = get_pdsync_file.data
    df_test = get_test_file.data
    df_test = df_test.loc[(df_test['pipeline'] == 'Opportunities')]
    df_test=df_test.loc[(df_test['channel_influence'] != 'HCLS-Alpha')]
    df_test=df_test.loc[(df_test['channel_influence'] == 'GCP')]
    df_test=df_test.loc[(df_test['stage_id'].isna()==False)]
    df_test=df_test.loc[(df_test['stage_id']!='deleted')]
    df_test=df_test.loc[(df_test['stage_id']!='DA')]
    index_stage = df_test['stage_id'].isin(['On Hold'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['stage_id']] = 'onhold'
    index_stage = df_test['last_stage_id'].isin(['On Hold'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'onhold'

    index_stage = df_test['last_stage_id'].isin(['On HOLD'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'onhold'
    index_stage = df_test['last_stage_id'].isin(['Discovery Meeting'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'discovery_meeting'

    index_stage = df_test['last_stage_id'].isin(['Problem Definition Finalized'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'problem_definition_finalized'

    index_stage = df_test['last_stage_id'].isin(['Proposal Shared'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'proposal_shared'

    index_stage = df_test['last_stage_id'].isin(['Pre-Commit'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'pre_Commit'

    index_stage = df_test['last_stage_id'].isin(['Committed'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'committed'
    
    
    df_test['close_time'] = df_test['close_time'].astype('datetime64[ns]')
    df_test['deal_created'] = df_test['deal_created'].astype('datetime64[ns]')
    df_test['update_time'] = df_test['update_time'].astype('datetime64[ns]')
    df_test['last_stage_change_time'] = df_test['last_stage_change_time'].astype('datetime64[ns]')
    df_test['deal_duration'] = df_test['close_time'] -  df_test['deal_created']
    df_test['deal_duration_days'] = (df_test['deal_duration']).dt.days
    df_test['deal_value'] = pd.to_numeric(df_test['deal_value'],errors = 'coerce')
    df_test['done_activities_count'] = pd.to_numeric(df_test['done_activities_count'],errors = 'coerce')
    df_test['onhold'] = pd.to_numeric(df_test['onhold'],errors = 'coerce')
    df_test['discovery_meeting'] = pd.to_numeric(df_test['discovery_meeting'],errors = 'coerce')
    df_test['problem_definition_finalized'] = pd.to_numeric(df_test['problem_definition_finalized'],errors = 'coerce')
    df_test['proposal_shared'] = pd.to_numeric(df_test['proposal_shared'],errors = 'coerce')
    df_test['contracting'] = pd.to_numeric(df_test['contracting'],errors = 'coerce')
    df_test['committed'] = pd.to_numeric(df_test['committed'],errors = 'coerce')
    df_test['last_updated_by'] = pd.to_numeric(df_test['last_updated_by'],errors = 'coerce')
    df_test['weights'] = pd.to_numeric(df_test['weights'],errors = 'coerce')
    df_test['deal_duration_days'] = pd.to_numeric(df_test['deal_duration_days'],errors = 'coerce')
    
    
    df_test['last_stage_change_time_year'] = df_test['last_stage_change_time'].dt.year
    df_test['last_stage_change_time_month'] = df_test['last_stage_change_time'].dt.month
    df_test['last_stage_change_time_week'] = df_test['last_stage_change_time'].dt.week
    df_test['last_stage_change_time_day'] = df_test['last_stage_change_time'].dt.day
    df_test['update_time_year'] = df_test['update_time'].dt.year
    df_test['update_time_month'] = df_test['update_time'].dt.month
    df_test['update_time_week'] = df_test['update_time'].dt.week
    df_test['update_time_day'] = df_test['update_time'].dt.day
    df_test['deal_created_year'] = df_test['deal_created'].dt.year
    df_test['deal_created_month'] = df_test['deal_created'].dt.month
    df_test['deal_created_week'] = df_test['deal_created'].dt.week
    df_test['deal_created_day'] = df_test['deal_created'].dt.day
    
    conditions = [
    (df_test['stage_id'] == 'Discovery Meeting') & (df_test['last_stage_id'].isin(['discovery_meeting', 'proposal_shared','Contracting', 'problem_definition_finalized', 'pre_Commit'])),
    (df_test['stage_id'] == 'Discovery Meeting') & (~df_test['last_stage_id'].isin(['discovery_meeting', 'proposal_shared','Contracting', 'problem_definition_finalized', 'pre_Commit'])),
    (df_test['stage_id'] == 'Problem Definition Finalized') & (df_test['last_stage_id'].isin(['proposal_shared','Contracting', 'problem_definition_finalized', 'pre_Commit'])),
    (df_test['stage_id'] == 'Problem Definition Finalized') & (~df_test['last_stage_id'].isin(['proposal_shared','Contracting', 'problem_definition_finalized', 'pre_Commit'])),
    (df_test['stage_id'] == 'Proposal Shared') & (df_test['last_stage_id'].isin(['proposal_shared','Contracting', 'pre_Commit'])),
    (df_test['stage_id'] == 'Proposal Shared') & (~df_test['last_stage_id'].isin(['proposal_shared','Contracting', 'pre_Commit'])),
    (df_test['stage_id'] == 'Contracting') & (df_test['last_stage_id'].isin(['Contracting', 'pre_Commit'])),
    (df_test['stage_id'] == 'Contracting') & (~df_test['last_stage_id'].isin(['Contracting', 'pre_Commit'])),
    (df_test['stage_id'] == 'Pre-Commit') & (df_test['last_stage_id'].isin(['pre_Commit'])),
    (df_test['stage_id'] == 'Pre-Commit') & (~df_test['last_stage_id'].isin(['pre_Commit'])),
     ]

    st_values = [1,0,1,0,1,0,1,0,1,0]
    
    
    df_test['stage_id_mismatch'] = np.select(conditions, st_values)
    
    
    
    df_final = df_test[['business_category', 'committed', 'business_impact',
       'deal_created_year', 'done_activities_count', 'deal_created_month',
       'BU', 'stage_id', 'mega_deal', 'client_domain', 'stage_id_mismatch',
       'update_time_month', 'update_time_day', 'contracting',
       'last_stage_change_time_month', 'source_of_lead', 'update_time_year',
       'deal_value',
       'deal_duration_days','deal_id','status']]
    
    df_final = df_final[df_final['status'] == 'open']
    
    cat_col = list(df_final.select_dtypes(include='O').columns)
    
    df_final[cat_col]=df_final[cat_col].fillna(df_final.mode().iloc[0])
    
    labelencoder = LabelEncoder()


    df_final[cat_col] = df_final[cat_col].apply(LabelEncoder().fit_transform)

    df_final = df_final.astype('float64')
    
    
    deal_id = df_final['deal_id']
    status = df_final['status']
    
    display(create_download_link_l1gcp_formatted(df_final))
    
    df_final = df_final.drop(columns=['deal_id', 'status'])
    
    pred_classification = l1gcp_predict(df_final)
    
    df_final['class_pred'] = pred_classification
    
    deal_duration_days = df_final['deal_duration_days']
    
    df_final = df_final.drop(columns=['deal_duration_days'])
    
    pred_regression = l1gcp_predict_reg(df_final)
    
    df_final['predicted_days'] = pred_regression
    
    display(df_final)
    
    df_final['deal_id'] = deal_id
    
    df = pd.merge(df_final, df_test[["deal_id","deal_created", "sow_start_date", "sow_end_date"]], on="deal_id", how="right")
    
    df = df.dropna() 

    df['predicted_close_date']= df.apply(new_date, axis=1)
    
    df['predicted_close_monthyear'] = df.predicted_close_date.dt.to_period('M')
    
    df_csv = df[['deal_id', 'deal_value', 'deal_created', 'sow_start_date', 'sow_end_date','class_pred', 'predicted_close_date']]
    
    display(create_download_link_l1gcp_pred(df_csv))
    
    df_date = df[(df['predicted_close_monthyear'] > "2022-09")]
    
    deal_value_monthly =  df_date.groupby(['predicted_close_monthyear'], as_index=False)['deal_value'].sum()
    print('Predicted OBV for all deals month-wise:\n')
    time.sleep(2)
    display(deal_value_monthly)
    df_date_won = df_date[df_date['class_pred']==1.0]
    deal_value_won =  df_date_won.groupby(['predicted_close_monthyear','class_pred'], as_index=False)['deal_value'].sum()
    print('Predicted OBV for predicted won deals month-wise:\n')
    time.sleep(2)
    display(deal_value_won)
    won_deals = df_date_won.groupby('predicted_close_monthyear', as_index=False)['class_pred'].count()
    print('Predicted no. of won deals month-wise:\n')
    time.sleep(2)
    display(won_deals)
    deal_duration = df_date.groupby('predicted_close_monthyear', as_index=False)['predicted_days'].mean()
    print('Predicted avg. deal duration of all deals month-wise:\n')
    time.sleep(2)
    display(deal_duration)
    deal_duration_won = df_date_won.groupby('predicted_close_monthyear', as_index=False)['predicted_days'].mean()
    print('Predicted avg. deal duration of won deals month-wise:\n')
    time.sleep(2)
    display(deal_duration_won)
    
    

    

#-------------------------# Load pre-trained model L1 GCP#---------------------------------------#
def l1gcp_model():
    with open('./xgb_classification_l1gcp.pkl', 'rb') as f:
        l1gcp_model = pickle.load(f)
    return l1gcp_model


def l1gcp_predict(df):
         # Feature arragend as per the model
    model_l1gcp=l1gcp_model()
    predict_l1gcp = model_l1gcp.predict(df)               # Prediction using the model
#     prediction = np.sum(predict_l0, dtype=np.int32)      # Predicted result is made into readable format
    return predict_l1gcp
 
def l1gcp_model_reg():
    with open('./xgb_regression_l1gcp.pkl', 'rb') as f:
        l1gcp_model_reg = pickle.load(f)
    return l1gcp_model_reg


def l1gcp_predict_reg(df):
         # Feature arragend as per the model
    model_l1gcp_reg=l1gcp_model_reg()
    predict_l1gcp_reg = model_l1gcp_reg.predict(df)               # Prediction using the model
#     prediction = np.sum(predict_l0, dtype=np.int32)      # Predicted result is made into readable format
    return predict_l1gcp_reg    
#--------------------------------# Create downlod links L1-gcp retraining #-------------------------#
def create_download_link_l1gcp_retrain( df, title = "Download formatted test CSV file for L1-GCP", filename = "formatted-data-L1GCP.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'     
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

def create_download_link_l1gcp_pred_rt( df, title = "Download L0 prediction CSV file", filename = "prediction-L1GCP-pretrained.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'     
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)
#--------------------------------#L0 Format for re-training model#-------------------------------#

def new_date(row):
    return pd.to_datetime(row['deal_created'])+pd.DateOffset(days=int(row['predicted_days']))

def l1gcp_format_retrain():
    
    df_pdsync = get_pdsync_file.data
    df_test = get_test_file.data
    df_test = df_test.loc[(df_test['pipeline'] == 'Opportunities')]
    df_test=df_test.loc[(df_test['channel_influence'] != 'HCLS-Alpha')]
    df_test=df_test.loc[(df_test['channel_influence'] == 'GCP')]
    df_test=df_test.loc[(df_test['stage_id'].isna()==False)]
    df_test=df_test.loc[(df_test['stage_id']!='deleted')]
    df_test=df_test.loc[(df_test['stage_id']!='DA')]
    index_stage = df_test['stage_id'].isin(['On Hold'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['stage_id']] = 'onhold'
    index_stage = df_test['last_stage_id'].isin(['On Hold'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'onhold'

    index_stage = df_test['last_stage_id'].isin(['On HOLD'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'onhold'
    index_stage = df_test['last_stage_id'].isin(['Discovery Meeting'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'discovery_meeting'

    index_stage = df_test['last_stage_id'].isin(['Problem Definition Finalized'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'problem_definition_finalized'

    index_stage = df_test['last_stage_id'].isin(['Proposal Shared'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'proposal_shared'

    index_stage = df_test['last_stage_id'].isin(['Pre-Commit'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'pre_Commit'

    index_stage = df_test['last_stage_id'].isin(['Committed'])
    index_stage.value_counts()
    df_index=index_stage.index[index_stage]
    df_test.loc[df_index,['last_stage_id']] = 'committed'
    
    
    df_test['close_time'] = df_test['close_time'].astype('datetime64[ns]')
    df_test['deal_created'] = df_test['deal_created'].astype('datetime64[ns]')
    df_test['update_time'] = df_test['update_time'].astype('datetime64[ns]')
    df_test['last_stage_change_time'] = df_test['last_stage_change_time'].astype('datetime64[ns]')
    df_test['deal_duration'] = df_test['close_time'] -  df_test['deal_created']
    df_test['deal_duration_days'] = (df_test['deal_duration']).dt.days
    df_test['deal_value'] = pd.to_numeric(df_test['deal_value'],errors = 'coerce')
    df_test['done_activities_count'] = pd.to_numeric(df_test['done_activities_count'],errors = 'coerce')
    df_test['onhold'] = pd.to_numeric(df_test['onhold'],errors = 'coerce')
    df_test['discovery_meeting'] = pd.to_numeric(df_test['discovery_meeting'],errors = 'coerce')
    df_test['problem_definition_finalized'] = pd.to_numeric(df_test['problem_definition_finalized'],errors = 'coerce')
    df_test['proposal_shared'] = pd.to_numeric(df_test['proposal_shared'],errors = 'coerce')
    df_test['contracting'] = pd.to_numeric(df_test['contracting'],errors = 'coerce')
    df_test['committed'] = pd.to_numeric(df_test['committed'],errors = 'coerce')
    df_test['last_updated_by'] = pd.to_numeric(df_test['last_updated_by'],errors = 'coerce')
    df_test['weights'] = pd.to_numeric(df_test['weights'],errors = 'coerce')
    df_test['deal_duration_days'] = pd.to_numeric(df_test['deal_duration_days'],errors = 'coerce')
    
    
    df_test['last_stage_change_time_year'] = df_test['last_stage_change_time'].dt.year
    df_test['last_stage_change_time_month'] = df_test['last_stage_change_time'].dt.month
    df_test['last_stage_change_time_week'] = df_test['last_stage_change_time'].dt.week
    df_test['last_stage_change_time_day'] = df_test['last_stage_change_time'].dt.day
    df_test['update_time_year'] = df_test['update_time'].dt.year
    df_test['update_time_month'] = df_test['update_time'].dt.month
    df_test['update_time_week'] = df_test['update_time'].dt.week
    df_test['update_time_day'] = df_test['update_time'].dt.day
    df_test['deal_created_year'] = df_test['deal_created'].dt.year
    df_test['deal_created_month'] = df_test['deal_created'].dt.month
    df_test['deal_created_week'] = df_test['deal_created'].dt.week
    df_test['deal_created_day'] = df_test['deal_created'].dt.day
    
    conditions = [
    (df_test['stage_id'] == 'Discovery Meeting') & (df_test['last_stage_id'].isin(['discovery_meeting', 'proposal_shared','Contracting', 'problem_definition_finalized', 'pre_Commit'])),
    (df_test['stage_id'] == 'Discovery Meeting') & (~df_test['last_stage_id'].isin(['discovery_meeting', 'proposal_shared','Contracting', 'problem_definition_finalized', 'pre_Commit'])),
    (df_test['stage_id'] == 'Problem Definition Finalized') & (df_test['last_stage_id'].isin(['proposal_shared','Contracting', 'problem_definition_finalized', 'pre_Commit'])),
    (df_test['stage_id'] == 'Problem Definition Finalized') & (~df_test['last_stage_id'].isin(['proposal_shared','Contracting', 'problem_definition_finalized', 'pre_Commit'])),
    (df_test['stage_id'] == 'Proposal Shared') & (df_test['last_stage_id'].isin(['proposal_shared','Contracting', 'pre_Commit'])),
    (df_test['stage_id'] == 'Proposal Shared') & (~df_test['last_stage_id'].isin(['proposal_shared','Contracting', 'pre_Commit'])),
    (df_test['stage_id'] == 'Contracting') & (df_test['last_stage_id'].isin(['Contracting', 'pre_Commit'])),
    (df_test['stage_id'] == 'Contracting') & (~df_test['last_stage_id'].isin(['Contracting', 'pre_Commit'])),
    (df_test['stage_id'] == 'Pre-Commit') & (df_test['last_stage_id'].isin(['pre_Commit'])),
    (df_test['stage_id'] == 'Pre-Commit') & (~df_test['last_stage_id'].isin(['pre_Commit'])),
     ]

    st_values = [1,0,1,0,1,0,1,0,1,0]
    
    
    df_test['stage_id_mismatch'] = np.select(conditions, st_values)
    
    df_final = df_test[['business_category', 'committed', 'business_impact',
       'deal_created_year', 'done_activities_count', 'deal_created_month',
       'BU', 'stage_id', 'mega_deal', 'client_domain', 'stage_id_mismatch',
       'update_time_month', 'update_time_day', 'contracting',
       'last_stage_change_time_month', 'source_of_lead', 'update_time_year',
       'deal_value',
       'deal_duration_days','deal_id','status']]
    
    df_open = df_test[['business_category', 'committed', 'business_impact',
       'deal_created_year', 'done_activities_count', 'deal_created_month',
       'BU', 'stage_id', 'mega_deal', 'client_domain', 'stage_id_mismatch',
       'update_time_month', 'update_time_day', 'contracting',
       'last_stage_change_time_month', 'source_of_lead', 'update_time_year',
       'deal_value',
       'deal_duration_days','deal_id','status']]

    
    df_final = df_final[~(df_final['status'] == 'open')]
    
    df_open = df_open[df_open['status'] == 'open']
    
    cat_col = list(df_final.select_dtypes(include='O').columns)
    
    df_final[cat_col]=df_final[cat_col].fillna(df_final.mode().iloc[0])
    
    df_open[cat_col]=df_open[cat_col].fillna(df_open.mode().iloc[0])
    
    labelencoder = LabelEncoder()


    df_final[cat_col] = df_final[cat_col].apply(LabelEncoder().fit_transform)
    
    df_open[cat_col] = df_open[cat_col].apply(LabelEncoder().fit_transform)

    df_final = df_final.astype('float64')
    
    df_open = df_open.astype('float64')
    
    deal_id = df_final['deal_id']
    
    deal_id_open = df_open['deal_id']
    
    status = df_final['status']
    
    
    display(create_download_link_l1gcp_retrain(df_final))
    
    df_final = df_final.drop(columns=['deal_id'])
    
    df_open = df_open.drop(columns=['deal_id','status'])
    
    X = df_final.drop(columns=['status'])
    
    y = df_final['status']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

#     estimator = XGBClassifier(
#     objective= 'binary:logistic',
#     seed=42
#     )
    
#     params = {
#         'min_child_weight': [1,3, 5, 10],
#         'gamma': [0.5, 1, 1.5, 2, 5],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 1.0],
#         'max_depth': [2,3, 4, 5],
#         'learning_rate':[0.1, 0.01, 0.05, 0.001]
#         'n_estimators':[100,200,300,500,700,1000]
#         }
    
#     grid_search = GridSearchCV(
#     estimator=estimator,
#     param_grid=parameters,
#     scoring = 'roc_auc',
#     verbose=True
#     )
    
#     gcv_class = grid_search.fit(X, Y)
#     xgb_class = gcv.best_estimator_
    
    xgbmodel = XGBClassifier(learning_rate = 0.01, n_estimators=300, max_depth=5)
    xgbmodel.fit(X_train, y_train)
    
    pred_open = xgbmodel.predict(df_open)
    
    
    df_open['class_pred'] = pred_open
    
    deal_duration_days = df_final['deal_duration_days']
    
    X_reg = df_final.drop(columns=['deal_duration_days'])
    
    Y_reg = df_final['deal_duration_days']
    
    
    X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, Y_reg, test_size=0.10)
    
    reg_model = xgb.XGBRegressor(colsample_bytree = 0.3, learning_rate=0.01, max_depth=3, n_estimators= 1000)
    
    reg_model.fit(X_train_reg, y_train_reg)
    
    df_open = df_open.drop(columns=['deal_duration_days'])
    
    df_open= df_open.rename(columns={'class_pred': 'status'})
    
    df_open = df_open[['business_category', 'committed', 'business_impact',
       'deal_created_year', 'done_activities_count', 'deal_created_month',
       'BU', 'stage_id', 'mega_deal', 'client_domain', 'stage_id_mismatch',
       'update_time_month', 'update_time_day', 'contracting',
       'last_stage_change_time_month', 'source_of_lead', 'update_time_year',
       'deal_value',
       'status']]
    
    pred_open_reg = reg_model.predict(df_open)
    
    df_open['predicted_days'] = pred_open_reg
    
    display(df_open)
    
    
    
    df_open['deal_id'] = deal_id_open
    
    df = pd.merge(df_open, df_test[["deal_id","deal_created", "sow_start_date", "sow_end_date"]], on="deal_id", how="right")
    
    df = df.dropna() 

    df['predicted_close_date']= df.apply(new_date, axis=1)
    
    df['predicted_close_monthyear'] = df.predicted_close_date.dt.to_period('M')
    
    df= df.rename(columns={'status': 'class_pred'})
    
    df_csv = df[['deal_id', 'deal_value', 'deal_created', 'sow_start_date', 'sow_end_date', 'class_pred', 'predicted_close_date']]
    
    display(create_download_link_l1gcp_pred_rt(df_csv))
    
    df_date = df[(df['predicted_close_monthyear'] > "2022-09")]
    
    deal_value_monthly =  df_date.groupby(['predicted_close_monthyear'], as_index=False)['deal_value'].sum()
    print('Predicted OBV for all deals month-wise:\n')
    time.sleep(2)
    display(deal_value_monthly)
    df_date_won = df_date[df_date['class_pred']==1.0]
    deal_value_won =  df_date_won.groupby(['predicted_close_monthyear','class_pred'], as_index=False)['deal_value'].sum()
    print('Predicted OBV for predicted won deals month-wise:\n')
    time.sleep(2)
    display(deal_value_won)
    won_deals = df_date_won.groupby('predicted_close_monthyear', as_index=False)['class_pred'].count()
    print('Predicted no. of won deals month-wise:\n')
    time.sleep(2)
    display(won_deals)
    deal_duration = df_date.groupby('predicted_close_monthyear', as_index=False)['predicted_days'].mean()
    print('Predicted avg. deal duration of all deals month-wise:\n')
    time.sleep(2)
    display(deal_duration)
    deal_duration_won = df_date_won.groupby('predicted_close_monthyear', as_index=False)['predicted_days'].mean()
    print('Predicted avg. deal duration of won deals month-wise:\n')
    time.sleep(2)
    display(deal_duration_won)
         
#--------------------------------# Create downlod links L0 retraining #-------------------------#
def create_download_link_l0_retrain( df, title = "Download formatted test CSV file for L0", filename = "formatted-data-L0.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'     
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

def create_download_link_l0_pred_rt( df, title = "Download L0 prediction CSV file", filename = "prediction-L0-pretrained.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'     
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)    
        
#-------------------------------#Level Selector button pre-trained#-----------------------------------------#
lev_button = widgets.Button(
description='Format Data',
disabled=False,
button_style='info', 
tooltip='Run'
)
    # #     icon='play

label_level = widgets.Label('Please select level for prediction')
    
level_selector = widgets.RadioButtons(
                         options=['L0','L1-GCP'],
                         value=None,
                         description='Select Level: ',
                         disabled=False,
#     layout=widgets.Layout(width='100%')
)
text_l0select = 'L0 Level Selected'
l0select = widgets.HTML(value = f"<b><font color='#36b38d' size=3>{text_l0select}</b>")

text_l1gcpselect = 'L1-GCP Level Selected'
l1gcpselect = widgets.HTML(value = f"<b><font color='#36b38d' size=3>{text_l1gcpselect}</b>")
def selectlevel(button):

        selection = level_selector.get_interact_value()
        if (selection == "L0"):
                display(l0select)
                time.sleep(10)
                l0_format_pretrain()               
        elif (selection =="L1-GCP"):
#                 level1_selector.observe(on_change_l1)
                display(l1gcpselect)
                time.sleep(10)
                l1gcp_format_pretrain()
    
#-------------------------------#Level Selector button re-training#-----------------------------------------#
lev_button_rt = widgets.Button(
description='Format Data',
disabled=False,
button_style='info', 
tooltip='Run'
)
    # #     icon='play

label_level_rt = widgets.Label('Please select level for prediction')
    
level_selector_rt = widgets.RadioButtons(
                         options=['L0','L1-GCP'],
                         value=None,
                         description='Select Level: ',
                         disabled=False,
#     layout=widgets.Layout(width='100%')
)
text_l0select_rt = 'L0 Level Selected'
l0select = widgets.HTML(value = f"<b><font color='#36b38d' size=3>{text_l0select_rt}</b>")

text_l1gcpselect_rt = 'L1-GCP Level Selected'
l1gcpselect_rt = widgets.HTML(value = f"<b><font color='#36b38d' size=3>{text_l1gcpselect_rt}</b>")
def selectlevel_rt(button):

        selection = level_selector_rt.get_interact_value()
        if (selection == "L0"):
                display(l0select)
                time.sleep(10)
                l0_format_retrain()               
        elif (selection =="L1-GCP"):
                display(l1gcpselect_rt)
                time.sleep(10)
                l1gcp_format_retrain()    
#---------------------------------------# Training Method Selector #-------------------------------#    
selector = widgets.RadioButtons(
    options=['Predict with existing model', 'Re-train model'],
    value=None,
    description='',
    disabled=False
)

button = widgets.Button(
     description='Next',
     disabled=False,
     button_style='info', 
     tooltip='Run' # 'success', 'info', 'warning', 'danger' or ''
)


def evaluate(button):

    selection = selector.get_interact_value()

    if (selection == "Predict with existing model"):
        pdsync_upload_button.on_click(get_pdsync_file)
        testfile_upload_button.on_click(get_test_file) 
        lev_button.on_click(selectlevel)

#             left_box_pdsywidgets.HBox([label_pdsync_test,uploader_pdsync_file,pdsync_upload_file_button])
        left_box_pdsync_upload=widgets.HBox([uploader_pdsync_file,pdsync_upload_button])
        left_box_test_upload=widgets.HBox([uploader_test_file,testfile_upload_button])
        left_box_level = widgets.VBox([level_selector, lev_button])
#         left_box_pdsync = widgets.VBox[(label_pdsync_test,left_box_pdsync_upload)]
        display(label_pdsync_test,left_box_pdsync_upload,label_test, left_box_test_upload, left_box_level)

    elif (selection == "Re-train model"):
        pdsync_upload_button.on_click(get_pdsync_file)
        testfile_upload_button.on_click(get_test_file) 
        lev_button_rt.on_click(selectlevel_rt)
        left_box_pdsync_upload=widgets.HBox([uploader_pdsync_file,pdsync_upload_button])
        left_box_test_upload=widgets.HBox([uploader_test_file,testfile_upload_button])
        left_box_level_rt = widgets.VBox([level_selector_rt, lev_button_rt])
#         left_box_pdsync = widgets.VBox[(label_pdsync_test,left_box_pdsync_upload)]
        display(label_pdsync_test,left_box_pdsync_upload,label_test, left_box_test_upload, left_box_level_rt)
    else:
        print('Error!')
        selector.value = None     

button.on_click(evaluate)        
text_training_method = 'Select prediction method'
training_method_label = widgets.HTML(value = f"<b><font size=4>{text_training_method}</b>")
left_box = widgets.VBox([training_method_label,selector, button])
widgets.HBox([left_box])



Label(value='Please upload PD Sync file in .csv or .xlsx format. Please click the Next button after Upload cha…

Label(value='Please upload test file in .csv or .xlsx format. Please click the Next button after Upload change…

HTML(value="<b><font color='#36b38d' size=3>L1-GCP Level Selected</b>")

<ipython-input-61-120b34a89b97>:897: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_test['last_stage_change_time_week'] = df_test['last_stage_change_time'].dt.week
<ipython-input-61-120b34a89b97>:901: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_test['update_time_week'] = df_test['update_time'].dt.week
<ipython-input-61-120b34a89b97>:905: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_test['deal_created_week'] = df_test['deal_created'].dt.week


[06:45:47] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

Predicted avg. deal duration of all deals month-wise:



,predicted_close_monthyear,predicted_days
0,2022-10,73.728592
1,2022-11,99.011131
2,2022-12,211.967438
3,2023-02,245.867950


Predicted avg. deal duration of won deals month-wise:



,predicted_close_monthyear,predicted_days
0,2022-10,79.016121
1,2022-12,211.967438
2,2023-02,245.867950


HTML(value="<b><font color='#36b38d' size=3>L1-GCP Level Selected</b>")